<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 17**
---



# **Modelos del lenguaje basados en redes neuronales artificiales**

---

### **Práctica: implementación del mecanismo de auto-atención con enmascaramiento del modelo Transformer**

Vamos a implementar el mecanismo de auto-atención con enmascaramiento del modelo Transformer en Pytorch. Para ello, vamos a seguir los pasos descritos anteriormente y suponer que ya tenemos las matrices de consultas (Q), claves (K) y valores (V) para cada token en la secuencia.


In [92]:
import torch
import numpy as np

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

Primero realizamos la multiplicación de las matrices Q y K

In [93]:
import numpy as np
Q_K = torch.matmul(Q, K.T)

Luego dividimos cada puntuación por la raíz cuadrada de la dimensión de los vectores K

In [94]:
d_k = K.size(-1)
Q_K /= np.sqrt(d_k)

Aplicamos la matriz de enmascaramiento

In [95]:
mask = torch.tensor([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 1]], dtype=torch.float32)

masked_scores = Q_K * mask

masked_scores[mask == 0] = -np.inf 

Aplicamos softmax

In [96]:
attention_weights = torch.softmax(masked_scores, dim=-1)

Multiplicamos con la matriz de valores (V) y mostramos el resultado:

In [ ]:
output = torch.matmul(attention_weights, V)
print(output)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

Una vez realizado el resultado debe ser:

<table>
<tr>
<td><b>z1</b></td><td>1.0000</td><td>0.0000</td><td>0.0000</td>
</tr>
<tr>
<td><b>z2</b></td><td>0.4568</td><td>0.5432</td><td>0.0000</td>
</tr>
<tr>
<td><b>z3</b></td><td>0.3219</td><td>0.3332</td><td>0.3449</td>
</tr>
<tr>
<td><b>z4</b></td><td>0.2309</td><td>0.5130</td><td>0.5260</td>
</tr>
</table>

#### **Objetivos de la práctica**

- Entender con detalle el funcionamiento del mecanismo de auto-atención con enmascaramiento.
- Practicar las operaciones matriciales en PyTorch.

---

## **Layer Normalization**


"Layer Normalization" es una técnica de normalización utilizada en el entrenamiento de redes neuronales profundas, introducida por Jimmy Lei Ba, Jamie Ryan Kiros y Geoffrey Hinton en 2016 (https://arxiv.org/pdf/1607.06450.pdf). Esta técnica es fundamental para mejorar la estabilidad y acelerar el entrenamiento de modelos de aprendizaje profundo, especialmente en el contexto de redes neuronales recurrentes (RNN) y modelos de atención como los Transformers.

### **Conceptos clave de la Layer Normalization:**

1. **Normalización por Ejemplo**: La Layer Normalization normaliza los datos a lo largo de las características para cada ejemplo individual en un lote. Esto significa que, para cada dato en el lote, la normalización se realiza calculando la media y la varianza de todas sus características.

2. **Cálculo de la Media y la Varianza**: Para cada ejemplo, se calcula la media y la varianza de todas sus características. Estos valores se utilizan para normalizar las características de ese ejemplo específico.

3. **Proceso de Normalización**: La normalización se realiza restando la media y dividiendo por la desviación estándar (raíz cuadrada de la varianza) de las características de cada ejemplo. Se añade un pequeño valor, conocido como epsilon, al denominador para evitar la división por cero.

4. **Parámetros Aprendibles**: Después de normalizar las características, se aplican dos parámetros aprendibles a cada una de ellas: un factor de escala (gamma) y un término de desplazamiento (beta). Estos parámetros son específicos para cada capa de la red y se ajustan durante el proceso de entrenamiento.

5. **Aplicaciones en Modelos de Secuencia**: La Layer Normalization ha demostrado ser particularmente efectiva en modelos que procesan datos secuenciales, como las redes neuronales recurrentes y los modelos basados en el mecanismo de atención, donde ayuda a mejorar la estabilidad y eficiencia del entrenamiento.

### **Ventajas de la Layer Normalization:**

- **Estabilidad en el Entrenamiento**: Contribuye a la estabilidad del entrenamiento de redes neuronales profundas, lo que es crucial para lograr un buen rendimiento en tareas complejas.
- **Eficiencia en Modelos de Secuencia**: Facilita el entrenamiento eficiente de modelos que trabajan con datos secuenciales, mejorando tanto la velocidad de convergencia como la calidad del modelo entrenado.
- **Flexibilidad en el Tamaño del Lote**: Al normalizar cada ejemplo de forma independiente, la Layer Normalization no depende del tamaño del lote, lo que la hace flexible para diferentes configuraciones de entrenamiento.

### **Representación matemática:**

La Layer Normalization se centra en normalizar las activaciones dentro de una capa para cada ejemplo de datos. A continuación se describe el proceso matemáticamente:

Dado un tensor de entrada $ X $ para una capa específica en la red, donde $ X $ tiene dimensiones $[N, F]$ (con $ N $ siendo el tamaño del lote y $ F $ el número de características o neuronas en la capa), la Layer Normalization se realiza de la siguiente manera para cada ejemplo $ n $ en el lote:

1. **Calcular la Media**:
   $$ \mu_n = \frac{1}{F} \sum_{f=1}^{F} x_{nf} $$
   Aquí, $ \mu_n $ es la media de las activaciones para el ejemplo $ n $.

2. **Calcular la Varianza**:
   $$ \sigma_n^2 = \frac{1}{F} \sum_{f=1}^{F} (x_{nf} - \mu_n)^2 $$
   Donde $ \sigma_n^2 $ es la varianza de las activaciones para el ejemplo $ n $.

3. **Normalizar**:
   $$ \hat{x}_{nf} = \frac{x_{nf} - \mu_n}{\sqrt{\sigma_n^2 + \epsilon}} $$
   Cada activación $ x_{nf} $ se normaliza restando la media $ \mu_n $ y dividiendo por la raíz cuadrada de la varianza $ \sigma_n^2 $, con un pequeño número $ \epsilon $ añadido para la estabilidad numérica (evitar la división por cero).

4. **Aplicar Parámetros Aprendibles**:
   $$ y_{nf} = \gamma \hat{x}_{nf} + \beta $$
   Finalmente, se aplica una transformación lineal a las activaciones normalizadas, donde $ \gamma $ y $ \beta $ son parámetros aprendibles específicos de la capa. Estos parámetros permiten que la normalización se ajuste de manera flexible durante el entrenamiento.

En esta formulación, $ n $ indexa los ejemplos en el lote y $ f $ indexa las características. Cada ejemplo se normaliza de forma independiente. Los parámetros $ \gamma $ y $ \beta $ son de la misma dimensión que el número de características $ F $, permitiendo una escala y un desplazamiento distintos para cada característica.

La Layer Normalization es especialmente útil en modelos de secuencias como los Transformers y las RNN, donde normaliza las activaciones a lo largo de las características para cada paso de tiempo o posición en la secuencia, contribuyendo a un entrenamiento más estable y eficiente.

---

## **Ejercicio: Implementación de la normalización de capa**

Asumiento los parámetros $\gamma$ y $\beta$ como 1 y 0 respectivamente, desarrolla un código que normalice el siguiente tensor de entrada:


In [20]:
import torch

my_tensor = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]], dtype=torch.float32)

nl = torch.nn.LayerNorm((2, 2), eps=1e-05, elementwise_affine=False)

print("Tamaño del lote:", my_tensor.shape[0])
print("Tamaño de la secuencia:", my_tensor.shape[1])
print("Tamaño de la entrada:", my_tensor.shape[2])
print()

print(nl(my_tensor))

Tamaño del lote: 3
Tamaño de la secuencia: 2
Tamaño de la entrada: 2

tensor([[[-1.3416, -0.4472],
         [ 0.4472,  1.3416]],

        [[-1.3416, -0.4472],
         [ 0.4472,  1.3416]],

        [[-1.3416, -0.4472],
         [ 0.4472,  1.3416]]])


In [ ]:
import numpy as np

my_tensor = my_tensor.numpy()
mean = np.mean(my_tensor, axis=(1, 2), keepdims=True)
std = np.std(my_tensor, axis=(1, 2), keepdims=True)
print((my_tensor - mean) / (std + 1e-05))

[[[-1.3416288 -0.4472096]
  [ 0.4472096  1.3416288]]

 [[-1.3416288 -0.4472096]
  [ 0.4472096  1.3416288]]

 [[-1.3416288 -0.4472096]
  [ 0.4472096  1.3416288]]]
